# XGBoost regression (multi-node with Dask)

<table>
    <tr>
        <td>
            <img src="../_img/dask-horizontal.svg" width="300">
        </td>
        <td>
            <img src="../_img/xgboost.png" width="300">
        </td>
        <td>
            <img src="../_img/snowflake.png" width="450">
        </td>
    </tr>
</table>

This notebook describes a machine learning training workflow using the famous [NYC Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). That dataset contains information on taxi trips in New York City.

In this exercise, you'll load data from Snowflake into a [Dask DataFrame](https://docs.dask.org/en/latest/dataframe.html) and use [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html) to answer this question

> based on characteristics that can be known at the beginning of a trip, what tip will this trip earn (as a % of the total fare)?

This notebook gives an introductory tutorial on how to use Dask to scale training of XGBoost models. For more detailed information, see ["Distributed XGBoost with Dask"](https://xgboost.readthedocs.io/en/latest/tutorials/dask.html) in the XGBoost documentation and ["XGBoost Training with Dask"](https://www.saturncloud.io/docs/tutorials/xgboost/) in Saturn Cloud's documentation.

<hr>

## Initialize A Dask Cluster

This tutorial uses multiple machines to show how to apply more computing resources to machine learning training. This is done with Dask. Saturn Cloud offers managed Dask clusters, which can be provisioned and modified programmatically.

The code below creates a Dask cluster using [`dask-saturn`](https://github.com/saturncloud/dask-saturn), the official Dask client for Saturn Cloud. It creates a cluster with the following specs:

* `n_workers=3` --> 3 machines in the cluster
* `scheduler_size='medium'` --> the Dask scheduler will have 4GB of RAM and 2 CPU cores
* `worker_size='large'` --> each worker machine will have 2 CPU cores and 16GB of RAM

To see a list of possible sizes, run the code below.

In [ ]:
import dask_saturn

dask_saturn.describe_sizes()

The `dask-saturn` code below creates two important objects: a cluster and a client.

* `cluster`: knows about and manages the scheduler and workers
    - can be used to create, resize, reconfigure, or destroy those resources
    - knows how to communicate with the scheduler, and where to find logs and diagnostic dashboards
* `client`: tells the cluster to do things
    - can send work to the cluster
    - can restart all the worker processes
    - can send data to the cluster or pull data back from the cluster

In [ ]:
import dask
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(
    n_workers=n_workers,
    scheduler_size="medium",
    worker_size="large",
)
client = Client(cluster)

If you created your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster in the Saturn UI if you want to get a head start!

In [ ]:
client.wait_for_workers(n_workers=n_workers)

### Monitor Resource Usage

This tutorial aims to teach you how to take advantage of multiple GPUs for data science workflows. To prove to yourself that Dask RAPIDS are utilizing the GPUs, it's important to understand how to monitor that utilization while your code is running. If you already know how to do that, skip to the next section.

Print the `cluster` object in a notebook renders a widget that shows the number of workers, available CPU and memory, and a dashboard link.

In [ ]:
cluster

Click that dashboard link to view some diagnostic information about the Dask cluster. This can be used to view the current resource utilization of workers in the cluster and lots of information about what they're currently working on.

<hr>

## Connect to Snowflake

This examples uses data stored in a Snowflake data warehouse. The code below expects the following environment variables to be set. If you haven't set these, return to the Saturn project page to set them.

* `SNOWFLAKE_ACCOUNT`
* `SNOWFLAKE_USER`
* `SNOWFLAKE_PASSWORD`
* `SNOWFLAKE_WAREHOUSE`
* `TAXI_DATABASE`
* `TAXI_SCHEMA`

For more details on these environment variables, see ["Connecting to Snowflake"](https://docs.snowflake.com/en/user-guide/python-connector-example.html#connecting-to-snowflake) in the `snowflake-connector-python` docs.

The `SNOWFLAKE_*` variables should be set up as Saturn credentials. The `TAXI_*` variables can be set on your Jupyter server or overwritten below based on the Snowflake warehouse and schema you used when running [load-data.sql](../snowflake/load-data.sql). Note that in order to update environment variables your Jupyter server will need to be stopped.

In [ ]:
import os
import dask.dataframe as dd
import snowflake.connector

SNOWFLAKE_ACCOUNT = os.environ["SNOWFLAKE_ACCOUNT"]
SNOWFLAKE_USER = os.environ["SNOWFLAKE_USER"]
SNOWFLAKE_PASSWORD = os.environ["SNOWFLAKE_PASSWORD"]

SNOWFLAKE_WAREHOUSE = os.environ["SNOWFLAKE_WAREHOUSE"]
TAXI_DATABASE = os.environ["TAXI_DATABASE"]
TAXI_SCHEMA = os.environ["TAXI_SCHEMA"]

conn_info = {
    "account": SNOWFLAKE_ACCOUNT,
    "user": SNOWFLAKE_USER,
    "password": SNOWFLAKE_PASSWORD,
    "warehouse": SNOWFLAKE_WAREHOUSE,
    "database": TAXI_DATABASE,
    "schema": TAXI_SCHEMA,
}
conn = snowflake.connector.connect(**conn_info)

<hr>

## Load data

This example is designed to run quickly with small resources. So let's just load a single month of taxi data for training.

This example uses Snowflake to handle the hard work of creating new features, then creates a Dask DataFrame with the result. It uses `dask.delayed()` to load partitions of the query result in parallel on the worker, so that you don't have to load the entire query result in this Jupyter and then send it to the Dask cluster.

The `load()` function specifies how each partition of the Dask dataframe should load its chunk of the data. As each partition of a Dask dataframe is a Pandas dataframe, we utilize the `fetch_pandas_all()` method.  We use a [binding for the Snowflake query](https://docs.snowflake.com/en/user-guide/python-connector-example.html#binding-data) so that we can pass different values at execution time. In this case, we pull one day of taxi rides into each partition.

In [ ]:
query = """
SELECT * FROM (
    SELECT
        pickup_taxizone_id,
        dropoff_taxizone_id,
        passenger_count,
        DIV0(tip_amount, fare_amount) AS tip_fraction,
        DAYOFWEEKISO(pickup_datetime) - 1 AS pickup_weekday,
        WEEKOFYEAR(pickup_datetime) AS pickup_weekofyear,
        HOUR(pickup_datetime) AS pickup_hour,
        (pickup_weekday * 24) + pickup_hour AS pickup_week_hour,
        MINUTE(pickup_datetime) AS pickup_minute
    FROM taxi_yellow
    WHERE
        DATE(pickup_datetime) = %s
) SAMPLE (30)
"""


@dask.delayed
def load(conn_info, query, day):
    with snowflake.connector.connect(**conn_info) as conn:
        taxi = conn.cursor().execute(query, str(day)).fetch_pandas_all()
        taxi.columns = taxi.columns.str.lower()
        return taxi

Run a query to determine which specific dates we need to pull

In [ ]:
def get_dates(start, end):
    date_query = """
    SELECT
        DISTINCT(DATE(pickup_datetime)) as date
    FROM taxi_yellow
    WHERE
        pickup_datetime BETWEEN %s and %s
    """
    dates_df = conn.cursor().execute(date_query, (start, end)).fetch_pandas_all()
    return dates_df["DATE"].tolist()


dates = get_dates("2019-01-01", "2019-01-31")

The `dd.from_delayed` function takes delayed Pandas outputs from a list and converts that to a high-level DataFrame object.

Since we're sampling in the Snowflake query, we run `persist()` to tell Dask to load all results into memory across the cluster. This ensures that Dask's lazy evaluation doesn't call the Snowflake query multiple times.

In [ ]:
taxi = dd.from_delayed([load(conn_info, query, day) for day in dates])
taxi = taxi.persist()
_ = wait(taxi)

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e6} MB")

In [ ]:
numeric_feat = [
    "pickup_weekday",
    "pickup_weekofyear",
    "pickup_hour",
    "pickup_week_hour",
    "pickup_minute",
    "passenger_count",
]
categorical_feat = [
    "pickup_taxizone_id",
    "dropoff_taxizone_id",
]
features = numeric_feat + categorical_feat
y_col = "tip_fraction"

taxi_train = taxi[features + [y_col]].astype(float).fillna(-1)

In [ ]:
taxi_train.head()

<hr>

## Train a model

This example uses the native Dask integration built into XGBoost. That integration was added in `xgboost` 1.3.0, and should be preferred to [`dask-xgboost`](https://github.com/dask/dask-xgboost).

In [ ]:
import xgboost as xgb

Training data for `xgboost.dask` needs to be prepared in a special object called `DaskDMatrix`. This is like the XGBoost `DMatrix` that you might be familiar with, but is backed by Dask's distributed collections (Dask DataFrame and Dask Array).

In [ ]:
data = taxi_train[features]
label = taxi_train[y_col]

dtrain = xgb.dask.DaskDMatrix(client=client, data=data, label=label)

You can pass any [xgboost parameters](https://xgboost.readthedocs.io/en/latest/parameter.html) to `xgb.dask.train()`. The training process will then start up on all workers that have some of the data in `dtrain`.

In [ ]:
%%time
result = xgb.dask.train(
    client=client,
    params={
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "learning_rate": 0.1,
        "max_depth": 5,
    },
    dtrain=dtrain,
    num_boost_round=50,
)

`xgb.dask.train()` produces a regular `xgb.core.Booster` object, the same model object produced by non-Dask training.

In [ ]:
booster = result["booster"]
type(booster)

# xgboost.core.Booster

<hr>

## Save model

`xgb.dask.train()` produces a regular `xgb.core.Booster` object, the same model object produced by non-Dask training.

In [ ]:
booster = result["booster"]
type(booster)

# xgboost.core.Booster

Once you've trained a model, save it in a file to use later for scoring or for comparison with other models.

There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/xgboost_dask.pkl", "wb") as f:
    cloudpickle.dump(booster, f)

<hr>

## Calculate metrics on test set

Use a different month for test set.

In [ ]:
test_dates = get_dates("2019-02-01", "2019-02-28")
taxi_test = dd.from_delayed([load(conn_info, query, day) for day in test_dates])
taxi_test = taxi_test.persist()
_ = wait(taxi_test)

`xgboost.dask.predict()` can be used to create predictions on a Dask collection using an XGBoost model object. Because the model object here is just a regular XGBoost model, using `dask-xgboost` for batch scoring doesn't require that you also perform training on Dask.

This function returns a Dask Array or Dask Series of predictions, depending on the input type.

In [ ]:
preds = xgb.dask.predict(client=client, model=booster, data=taxi_test[features])

The metrics functions in `dask_ml` can compute metrics on Dask collections like Dask Array and Dask DataFrame, so you never have to hold all of the test data in memory on the client. These functions intentionally mimic the metrics functions in scikit-learn.

In [ ]:
from dask_ml.metrics import mean_squared_error

mean_squared_error(taxi_test[y_col].to_dask_array(), preds.to_dask_array(), squared=False)

<hr>

## Next Steps

In this tutorial, you learned how to use Dask and `xgboost` for distributed model training.

If you want to try working with larger datasets, change the code in this notebook to use more data or explore how to programmatically resize your Dask cluster by following the tips in ["Managing Dask Resources for Machine Learning Training"](https://www.saturncloud.io/docs/tips-and-tricks/training-on-dask).

Alternatively, try [this notebook](./dashboard.ipynb) to learn how to use Saturn Cloud to deploy an interactive dashboard that includes a component which uses the models trained above.

<hr>